<a href="https://colab.research.google.com/github/google/applied-machine-learning-intensive/blob/master/v2/06_other_models/01_k_means/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2020 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# k-means


k-means clustering is an *unsupervised* machine learning algorithm that can be used to group items into clusters.

So far we have only worked with supervised algorithms. Supervised algorithms have training data with labels that identify the numeric value or class for each item. These algorithms use labeled data to build a model that can be used to make predictions.

k-means clustering is different. The training data is not labeled. Unlabeled training data is fed into the model, which attempts to find relationships in the data and create clusters based on those relationships. Once these clusters are formed, predictions can be made about which cluster new data items belong to.

The clusters can't easily be labeled in many cases. The clusters are "emergent clusters", which are created by the algorithm and don't always map to groupings that you might expect.

## Example: Groups of Mushrooms

Let's start by looking at a real world use case involving mushrooms. The University of California Irvine has a [dataset containing various attributes of mushrooms](https://www.kaggle.com/uciml/mushroom-classification). One of those attributes is the edibility of the mushroom: Is it edible or is it poisonous?  We want to see if we can find clusters of mushroom attributes that can be used to determine if a mushroom is edible or not.

### Load the data

For this example we'll load the [Mushroom Classification](https://www.kaggle.com/uciml/mushroom-classification) data. The dataset attributes about over `8,000` different mushrooms.

Upload your `kaggle.json` file and run the code block below.

In [0]:
! chmod 600 kaggle.json && (ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle) && mv kaggle.json ~/.kaggle/ && echo 'Done'

And then use the Kaggle API to download the dataset.

In [0]:
! kaggle datasets download uciml/mushroom-classification
! ls

Unzip the data.

In [0]:
! unzip mushroom-classification.zip
! ls

And finally, load the training data into a `DataFrame`.

In [0]:
import pandas as pd

data = pd.read_csv('mushrooms.csv')
data.sample(n=10)

### Exploratory Data Analysis

Let's take a closer look at the data that we'll be working with, starting with a simple describe.

In [0]:
data.describe(include='all')

It doesn't look like any columns are missing data since we see counts of `8,124` for every column.

It does look like all of the data is categorical. We'll need to convert it into numeric values for the model to work. Let's do it for every column except `class`. We aren't trying to predict class, but we do want to see if we can get pure clusters of one type of class. So we don't want it included in our training data. Also, it is the only feature that isn't observable without having dire consequences!

In [0]:
columns = [c for c in data.columns.values if c != 'class']
id_to_value_mappings = {}
value_to_id_mappings = {}

for column in columns:
  i_to_v = sorted(data[column].unique())
  v_to_i = { v:i for i, v in enumerate(i_to_v)}
  

  numeric_column = column + '-id'
  data[numeric_column] = [v_to_i[v] for v in data[column]]

  value_to_id_mappings[column] = v_to_i
  id_to_value_mappings[numeric_column] = i_to_v

numeric_columns = id_to_value_mappings.keys()
data[numeric_columns].describe()

### Perform Clustering

We now have numeric data that a model can handle. To run k-means clustering on the data, we simply load [k-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) from scikit-learn and ask the model to find a specific number of clusters for us.

Notice that we are scaling the data. The class IDs are integer values, and some columns have many more classes than others. Scaling helps make sure that columns with more classes don't have an undue influence on the model.

In [0]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale

model = KMeans(n_clusters=10)
model.fit(scale(data[numeric_columns]))

print(model.inertia_)

We asked scikit-learn to create 10 clusters for us, and then we printed out the `inertia_` for the resultant clusters. *Inertia* is the sum of the squared distances of samples to their closest cluster center. Typically, the smaller the inertia the better.

But why did we choose `10` clusters? And is the inertia that we received reasonable?

### Find the optimal number of clusters

With just one run of the algorithm, it is difficult to tell how many clusters we should have and what an appropriate inertia value is. k-means is trying to discover things about your data that you do not know. Picking a number of clusters at random isn't the best way to use k-means.

Instead, you should experiment with a few different cluster values and measure the inertia of each. As you increase the number of clusters, your inertia should decrease.

In [0]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt

clusters = list(range(5, 50, 5))
inertias = []

scaled_data = scale(data[numeric_columns])

for c in clusters:
  model = KMeans(n_clusters=c)
  model = model.fit(scaled_data)
  inertias.append(model.inertia_)

plt.plot(clusters, inertias)
plt.show()

The resulting graph should start high and to the left and curve down as the number of clusters grows. The initial slope is steep, but begins to level off. Your optimal number of clusters is somewhere in the ["elbow" of the graph](https://en.wikipedia.org/wiki/Elbow_method_(clustering)), as the slope levels.

Once you have this number, you need to then check to see if the number is reasonable for your use case. Say that the 'optimal' number of clusters for our customer segmentation is 15. Is that a reasonable number of clusters to deal with? If we have too many, we can overfit and make the model poor at generalizing. And what are the purposes of the clusters? If you are clustering mushrooms and want to find clusters that are definitely safe to eat, 15 or more clusters might be perfectly fine. If you are clustering customers for different advertising campaigns, 15 different campaigns might be more than your marketing department can handle.

Clustering the data is often just the start of your journey. Once you have clusters, you'll need to look at each group and try to determine what makes them similar. What patterns did the clustering find? And will that clustering be useful to you?

### Examining Clusters

Let's say that `15` is a reasonable number of clusters. We can rebuild the model using that setting.

In [0]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale

model = KMeans(n_clusters=15)
model.fit(scale(data[numeric_columns]))

print(model.inertia_)

Now let's see if we have any 'pure' clusters. These are clusters with all-edible or all-poisonous mushrooms.

In [0]:
import numpy as np

for cluster in sorted(np.unique(model.labels_)):
  num_edible = np.sum(data[model.labels_ == cluster]['class'] == 'e')
  total = np.sum(model.labels_ == cluster)
  print(cluster, num_edible / total)

In our model we had clusters `0`, `1`, `6`, and `10` be `100%` edible. Clusters `2`, `4`, `7`, and `12` were all poisonous. The remaining were a mix of the two.

Knowing this, let's look at one of the all-edible clusters and see what attributes we could look for to have confidence that we have an edible mushroom.

In [0]:
edible = data[model.labels_ == 1]

for column in edible.columns:
  if column.endswith('-id'):
    continue
  print(column, edible[column].unique())


The mapping of the letter codes to more descriptive text can be found in the [dataset description](https://www.kaggle.com/uciml/mushroom-classification).

## Example: Classification of Digits

Clustering for data exploration purposes can lead to interesting insights in to your data, but clustering can also be used for classification purposes.

In the example below, we'll try to use k-means clustering to predict handwritten digits.

### Load the data

We'll load the digits dataset packaged with scikit-learn.

In [0]:
from sklearn.datasets import load_digits

digits = load_digits()

### Scale the data

It is good practice to scale the data to ensure that outliers don't have too big of an impact on the clustering.

In [0]:
from sklearn.preprocessing import scale

scaled_digits = scale(digits.data)

### Fit a model

We can then create a k-means model with 10 clusters. (We know there are 10 digits from 0 through 9.)

In [0]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=10)
model = model.fit(scaled_digits)

### Make predictions

We can then use the model to predict which category a data point belongs to.

In the case below, we'll just use some of the data that we trained with for illustrative purposes. The prediction will provide a numeric value.

In [0]:
cluster = model.predict([scaled_digits[0]])[0]

cluster

What is this value? Is it the predicted digit?

No. This number is the cluster that the model thinks the digit belongs to. To determine the predicted digit, we'll need to see what other digits are in the cluster and choose the most popular one for our classification.


In [0]:
import numpy as np

labels = digits.target

cluster_to_digit = [
  np.argmax(
      np.bincount(
        np.array(
          [labels[i] for i in range(len(model.labels_)) if model.labels_[i] == cluster]
        )
      )
    ) for cluster in range(10)
]

cluster_to_digit

Here we can see the digit that each cluster represents.

### Measure model quality

If we do have labeled data, as is the case with our digits data, then we can measure the quality of our model using the [homogeneity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_score.html#sklearn.metrics.homogeneity_score_) score and the [completeness](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.completeness_score.html#sklearn.metrics.completeness_score) score.

In [0]:
from sklearn.metrics import homogeneity_score
from sklearn.metrics import completeness_score

homogeneity = homogeneity_score(labels, model.labels_)
completeness = completeness_score(labels, model.labels_)
homogeneity, completeness

# Exercises

## Exercise 1

Load the [iris dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html), create a k-means model with three clusters, and then find the homogeneity and completeness scores for the model. 

### **Student Solution**

In [0]:
# Your code goes here

---

### Answer Key

In [0]:
from sklearn.cluster import KMeans
from sklearn.datasets import load_iris
from sklearn.metrics import homogeneity_score
from sklearn.metrics import completeness_score
from sklearn.preprocessing import scale

import numpy as np

iris = load_iris()

scaled_iris = scale(iris.data)

model = KMeans(n_clusters=3)
model = model.fit(scaled_iris)

labels = iris.target

cluster_to_species = [
  np.argmax(
      np.bincount(
        np.array([
                  labels[i]
                  for i in range(len(model.labels_))
                  if model.labels_[i] == cluster
        ])
      )
    ) for cluster in range(3)
]

homogeneity = homogeneity_score(labels, model.labels_)
completeness = completeness_score(labels, model.labels_)
homogeneity, completeness

---

## Exercise 2

Load the [iris dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html), and then create a k-means model with three clusters using only two features. (Try to find the best two features for clustering.) Create a plot of the two features.

For each datapoint in the chart, use a [marker](https://matplotlib.org/api/markers_api.html) to encode the actual/correct species. For instance, use a triangle for Setosa, a square for Versicolour, and a circle for Virginica. Color each marker green if the predicted class matches the actual. Color each marker red if the classes don't match.

### **Student Solution**

In [0]:
# Your code goes here

---

### Answer Key

In [0]:
from sklearn.cluster import KMeans
from sklearn.datasets import load_iris
from sklearn.metrics import homogeneity_score
from sklearn.metrics import completeness_score
from sklearn.preprocessing import scale

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

iris = load_iris()

scaled_iris = scale(iris.data)
scaled_iris = np.array(scaled_iris)
labels = iris.target

highest_score, best_i, best_j = 0, 0, 0

for i in range(3):
  for j in range(3):
    if i != j:
      model = KMeans(n_clusters=3)
      model = model.fit(
          list(zip(np.array(iris.data)[:, i], np.array(iris.data)[:, j])))

      homogeneity = homogeneity_score(labels, model.labels_)
      completeness = completeness_score(labels, model.labels_)
      combined_score = homogeneity + completeness

      if combined_score > highest_score:
        highest_score = combined_score
        best_i = i
        best_j = j

cluster_to_species = [
  np.argmax(
      np.bincount(
        np.array([
                  labels[i]
                  for i in range(len(model.labels_))
                  if model.labels_[i] == cluster
        ])
      )
    ) for cluster in range(3)
]

correct = [cluster_to_species[model.labels_[i]] == labels[i] 
           for i in range(len(model.labels_))]

df = pd.DataFrame()
iris_data = np.array(iris.data)

df[iris.feature_names[best_i]] = iris_data[:, best_i]
df[iris.feature_names[best_j]] = iris_data[:, best_j]
df['species'] = labels
df['predicted_species'] = [cluster_to_species[model.labels_[i]] for i in range(len(model.labels_))]

MARKERS = ('^', 's', 'o')

fig, ax = plt.subplots(figsize=(10, 10))

for i in range(3):
  selector = (df['species'] == i) & (df['predicted_species'] == i)
  ax.scatter(
      df[selector][iris.feature_names[best_i]],
      df[selector][iris.feature_names[best_j]],
      marker = MARKERS[i],
      color = 'g',
      label = iris.feature_names[i] + ' correct'
  )
  selector = (df['species'] == i) & (df['predicted_species'] != i)
  ax.scatter(
      df[selector][iris.feature_names[best_i]],
      df[selector][iris.feature_names[best_j]],
      marker = MARKERS[i],
      color = 'r',
      label = iris.feature_names[i] + ' incorrect'
  )

ax.legend()
_ = plt.show()

---